In [1]:
# Import required libraries and dependencies
import warnings
warnings.filterwarnings("ignore", category=UserWarning)  # Stop KMeans memory leak warning

import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [3]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [4]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [5]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
crypto_data_scaled = StandardScaler().fit_transform(df_market_data[['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'price_change_percentage_1y']])

In [6]:
# Create a DataFrame with the scaled data
crypto_data_scaled_df = pd.DataFrame(crypto_data_scaled, columns=df_market_data.columns)

# Copy the crypto names from the original DataFrame
crypto_data_scaled_df["coin_id"] = df_market_data.index 

# Set the coin_id column as index
crypto_data_scaled_df = crypto_data_scaled_df.set_index("coin_id")

# Display the scaled DataFrame
crypto_data_scaled_df.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data.

In [7]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [9]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
for i in k:
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters    
    k_model = KMeans(n_clusters=i)
# 2. Fit the model to the data using `df_market_data_scaled`
    k_model.fit(crypto_data_scaled_df)
# 3. Append the model.inertia_ to the inertia list
    inertia.append(k_model.inertia_)

In [10]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_data)
elbow_df.head()

,k,inertia
0,1,287.000000
1,2,195.820218
2,3,123.190482
3,4,79.022435
4,5,69.668772


In [11]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
plot_elbow = elbow_df.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)
plot_elbow

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** Based on the elbow curve graph, k=4 seems to be the best choice to start the analysis.

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [12]:
# Initialise the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=0)

In [13]:
# Fit the K-Means model using the scaled DataFrame
model.fit(crypto_data_scaled_df)

KMeans(n_clusters=4, random_state=0)

In [14]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
crypto_data_clusters = model.predict(crypto_data_scaled_df)

# Print the resulting array of cluster values.
print(crypto_data_clusters)

[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 1 2 0 0 3
 0 0 0 0]


In [16]:
# Create a copy of the scaled DataFrame
crypto_data_predictions_df = crypto_data_scaled_df.copy()

In [17]:
# Add a new column to the DataFrame with the predicted clusters
crypto_data_predictions_df["PredictedClusters"] = crypto_data_clusters

# Display sample data
crypto_data_predictions_df.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,PredictedClusters
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2


In [22]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Colour the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
cluster_plot = crypto_data_predictions_df.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    hover_cols=["coin_id"],
    marker=["hex", "square", "cross", "inverted_triangle"],
    title = "The cryptocurrency represented by each data point",
    by="PredictedClusters"
)
cluster_plot

:NdOverlay   [PredictedClusters]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimise Clusters with Principal Component Analysis.

In [19]:
# Create a PCA model instance and set `n_components=3`.
pca=PCA(n_components=3)

In [20]:
# Use the PCA model with `fit_transform` to reduce the original scaled Dataframe
# down to three principal components.
crypto_data_pca = pca.fit_transform(crypto_data_scaled_df)

# View the first five rows of the DataFrame. 
crypto_data_pca[0:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [21]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
pca.explained_variance_ratio_

array([0.3719856 , 0.34700813, 0.17603793])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 89.4% is the total explained variance using just these 3 components. i.e., We dropped 10.6% of the variability by dropping 4 of the 7 components.

In [23]:
# Create a new DataFrame with the PCA data.
crypto_data_pca_df = pd.DataFrame(crypto_data_pca, columns=["PC1", "PC2", "PC3"])

# Copy the crypto names from the original scaled DataFrame
crypto_data_pca_df["coin_id"] = df_market_data.index

# Set the coin_id column as index
crypto_data_pca_df = crypto_data_pca_df.set_index("coin_id")

# Display the scaled PCA DataFrame
crypto_data_pca_df.head()

,PC1,PC2,PC3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [24]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [25]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
for i in k:
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters 
    model = KMeans(n_clusters=i, random_state=0)
# 2. Fit the model to the data using `df_market_data_pca`    
    model.fit(crypto_data_pca_df)
# 3. Append the model.inertia_ to the inertia list    
    inertia.append(model.inertia_)

In [26]:
# Create a dictionary with the data to plot the Elbow curve
elbow_crypto_data_pca = {
    "k": k,
    "inertia": inertia
}
# Create a DataFrame with the data to plot the Elbow curve
elbow_pca_df = pd.DataFrame(elbow_crypto_data_pca)

In [32]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
elbow_plot_pca = elbow_pca_df.hvplot.line(x="k", y="inertia", title="PCA Elbow Curve", xticks=k)
elbow_plot_pca

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:**  Both data show 4

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [28]:
# Initialise the K-Means model using the best value for k
pca4_model = KMeans(n_clusters=4, random_state=0)

In [29]:
# Fit the K-Means model using the scaled PCA DataFrame
pca4_model.fit(crypto_data_pca_df)

KMeans(n_clusters=4, random_state=0)

In [30]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
pca_4 = pca4_model.predict(crypto_data_pca_df)
# Print the resulting array of cluster values.
print(pca_4)

[2 2 0 0 2 2 2 2 2 0 0 0 0 2 0 2 0 0 2 0 0 2 0 0 0 0 0 0 2 0 0 0 1 2 0 0 3
 0 0 0 0]


In [31]:
# Create a copy of the scaled PCA DataFrame
pca4_prediction_df = crypto_data_pca_df.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
pca4_prediction_df['customer_segments'] = pca_4

# Display sample data
pca4_prediction_df.head()

,PC1,PC2,PC3,customer_segments
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,2
ethereum,-0.458261,0.458466,0.952877,2
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,2


In [33]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Colour the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
plot_pca = pca4_prediction_df.hvplot.scatter(
    title= 'PCA Clusters',
    x= 'PC1',
    y= 'PC2',
    by= 'customer_segments',
    hover_cols= 'Crypto_Names'
)
plot_pca

:NdOverlay   [customer_segments]
   :Scatter   [PC1]   (PC2)

### Visualise and Compare the Results

In this section, you will visually analyse the cluster analysis results by contrasting the outcome with and without using the optimisation techniques.

In [40]:
# Composite plot to contrast the Elbow curves
(plot_elbow.relabel("Elbow Curve") * elbow_plot_pca.relabel("PCA Elbow Curve"))


:Overlay
   .Curve.Elbow_Curve     :Curve   [k]   (inertia)
   .Curve.PCA_Elbow_Curve :Curve   [k]   (inertia)

In [34]:
# Composite plot to contrast the clusters
# use `+` to have two separate plots, as putting them together would be visually confusing
(cluster_plot + plot_pca).cols(1)

:Layout
   .NdOverlay.I  :NdOverlay   [PredictedClusters]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [customer_segments]
      :Scatter   [PC1]   (PC2)

#### Answer the following question: 

  * **Question:** After visually analysing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:**  The inertia decreased and the data points seem to be less dispersed. Having fewer features to cluster the data seems to reduce ambiguity, which leads to a better interpretation of the results. The best value for k is the same in both cases. The impact of using fewer features to cluster the data produces tighter clusters.